In [1]:
!pip install git+https://github.com/PennyLaneAI/pennylane

  Cloning https://github.com/PennyLaneAI/pennylane to /tmp/pip-req-build-ngjy5xfs
  Running command git clone -q https://github.com/PennyLaneAI/pennylane /tmp/pip-req-build-ngjy5xfs
     |████████████████████████████████| 4.4 MB 4.6 MB/s 
  Created wheel for PennyLane: filename=PennyLane-0.19.0.dev0-py3-none-any.whl size=674636 sha256=e61a1e64a9d95591e41a7c7ba91744614938c7128e2f8a25abf10e9af9caecb5
  Stored in directory: /tmp/pip-ephem-wheel-cache-5yrt0zp1/wheels/9e/4b/fe/27dcf8ba174161f9d1af1841251dc97013a33a66f16cd8d661
Successfully built PennyLane


In [1]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch import tensor

from pathlib import Path

import pennylane as qml
from pennylane import numpy as np
from pennylane.templates import embeddings as emb
from pennylane.templates import layers as lay

from typing import Union

import matplotlib.pyplot as plt

from music21 import converter, instrument, note, chord, stream
import glob, pickle, time

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [4]:
!wget https://github.com/theerfan/Maqenta/raw/main/data/notes.pk

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
# Midi.py

notes_dir = "notes.pk"


class Midi:
    def __init__(self, seq_length, device):
        self.seq_length = seq_length
        self.device = device

        if Path(notes_dir).is_file():
            self.notes = pickle.load(open(notes_dir, "rb"))
            # self.notes = pickle.loads(uploaded[notes_dir])
        else:
            self.notes = self.get_notes()
            pickle.dump(self.notes, open(notes_dir, "wb"))

        self.network_input, self.network_output = self.prepare_sequences(self.notes)
        print(f"Input shape: {self.network_input.shape}")
        print(f"Output shape: {self.network_output.shape}")

    def get_notes(self):
        """Get all the notes and chords from the midi files in the ./midi_songs directory"""
        # This is assuming that every interval between notes is the same (0.5)
        notes = []

        for file in glob.glob("midi_songs/*.mid"):
            midi = converter.parse(file)

            print("Parsing %s" % file)

            notes_to_parse = None

            try:  # file has instrument parts
                s2 = instrument.partitionByInstrument(midi)
                notes_to_parse = s2.parts[0].recurse()
            except:  # file has notes in a flat structure
                notes_to_parse = midi.flat.notes

            for element in notes_to_parse:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append(".".join(str(n) for n in element.normalOrder))

        with open(notes_dir, "wb") as filepath:
            pickle.dump(notes, filepath)

        return notes

    def prepare_sequences(self, notes):
        """Prepare the sequences used by the Neural Network"""
        self.n_vocab = len(set(notes))

        # get all pitch names
        pitchnames = sorted(set(item for item in notes))

        # create a dictionary to map pitches to integers
        self.note_to_int = {note: number for number, note in enumerate(pitchnames)}
        self.int_to_note = {number: note for number, note in enumerate(pitchnames)}

        network_input = []
        network_output = []

        # create input sequences and the corresponding outputs
        for i in range(len(self.notes) - self.seq_length):
            sequence_in = self.notes[i : i + self.seq_length]
            sequence_out = self.notes[i + self.seq_length]
            network_input.append([self.note_to_int[char] for char in sequence_in])
            network_output.append(self.note_to_int[sequence_out])

        n_patterns = len(network_input)

        # reshape the input into a format compatible with LSTM layers
        # So this is actuallyt (number of different inputs, sequence length, number of features)
        network_input = np.reshape(network_input, (n_patterns, self.seq_length))
        network_input = torch.tensor(network_input, device=self.device, dtype=torch.double)

        self.input_norms = torch.tensor(torch.linalg.norm(network_input, axis=1))
        
        # print(network_input.shape)
        for i in range(network_input.shape[0]):
            network_input[i] /= self.input_norms[i]
        # network_input = torch.div(network_input, self.input_norms)

        return (
            network_input,
            torch.tensor(network_output, device=self.device),
        )

    def create_midi_from_model(self, prediction_output, filename):
        """convert the output from the prediction to notes and create a midi file
        from the notes"""
        offset = 0
        output_notes = []

        # create note and chord objects based on the values generated by the model
        for pattern in prediction_output:
            # pattern is a chord
            if ("." in pattern) or pattern.isdigit():
                notes_in_chord = pattern.split(".")
                notes = []
                for current_note in notes_in_chord:
                    new_note = note.Note(int(current_note))
                    new_note.storedInstrument = instrument.Piano()
                    notes.append(new_note)
                new_chord = chord.Chord(notes)
                new_chord.offset = offset
                output_notes.append(new_chord)
            # pattern is a note
            else:
                new_note = note.Note(pattern)
                new_note.offset = offset
                new_note.storedInstrument = instrument.Piano()
                output_notes.append(new_note)

            # increase offset each iteration so that notes do not stack
            offset += 0.5

        midi_stream = stream.Stream(output_notes)

        midi_stream.write("midi", fp=filename)


In [4]:
# QLSTM.py

Embedding = Union[emb.AngleEmbedding, emb.AmplitudeEmbedding, emb.BasisEmbedding]
Layer = Union[
    lay.BasicEntanglerLayers,
    lay.ParticleConservingU1,
    lay.ParticleConservingU2,
    lay.RandomLayers,
    lay.StronglyEntanglingLayers,
]


class QLSTMCell(nn.Module):
    def quantum_op(
        self,
        wires,
        embedding: Embedding = emb.AmplitudeEmbedding,
        layer: Layer = lay.RandomLayers,
    ):
        def circuit_part(inputs, weights):
            if embedding == emb.AmplitudeEmbedding:
              embedding(inputs.cpu().detach(), wires=wires, normalize=True)
            else:
              embedding(inputs, wires=wires)
            if layer == lay.RandomLayers:
              seed = np.random.randint(1, 2**12)
              layer(weights, wires=wires, seed=seed)
            else:
              layer(weights, wires=wires)
            return [qml.expval(qml.PauliZ(wires=w)) for w in wires]

        return circuit_part

    def __init__(
        self,
        input_size,
        hidden_size,
        n_qubits=4,
        n_qlayers=1,
        dropout=0,
        batch_first=True,
        return_sequences=False,
        return_state=True,
        backend="default.qubit",
        device="cpu"
    ):
        super(QLSTMCell, self).__init__()
        self.n_inputs = input_size
        self.hidden_size = hidden_size
        self.concat_size = self.n_inputs + self.hidden_size
        self.n_qubits = n_qubits
        self.n_qlayers = n_qlayers
        self.backend = backend  # "default.qubit", "qiskit.basicaer", "qiskit.ibm"
        self.device = device # "cpu", "cuda"
        self.dropout = dropout

        self.batch_first = batch_first
        self.return_sequences = return_sequences
        self.return_state = return_state

        self.wires_forget = [f"wire_forget_{i}" for i in range(self.n_qubits)]
        self.wires_input = [f"wire_input_{i}" for i in range(self.n_qubits)]
        self.wires_update = [f"wire_update_{i}" for i in range(self.n_qubits)]
        self.wires_output = [f"wire_output_{i}" for i in range(self.n_qubits)]

        self.dev_forget = qml.device(self.backend, wires=self.wires_forget)
        self.dev_input = qml.device(self.backend, wires=self.wires_input)
        self.dev_update = qml.device(self.backend, wires=self.wires_update)
        self.dev_output = qml.device(self.backend, wires=self.wires_output)

        self.qlayer_forget = qml.QNode(
            self.quantum_op(self.wires_forget), self.dev_forget, interface="torch"
        )

        self.qlayer_input = qml.QNode(
            self.quantum_op(self.wires_input), self.dev_input, interface="torch"
        )

        self.qlayer_update = qml.QNode(
            self.quantum_op(self.wires_update), self.dev_update, interface="torch"
        )

        self.qlayer_output = qml.QNode(
            self.quantum_op(self.wires_output), self.dev_output, interface="torch"
        )

        weight_shapes = {"weights": (n_qlayers, n_qubits)}
        print(f"weight_shapes = (n_qlayers, n_qubits) = ({n_qlayers}, {n_qubits})")

        # self.clayer_in = torch.nn.Linear(self.concat_size, n_qubits)
        self.VQC_forget = qml.qnn.TorchLayer(self.qlayer_forget, weight_shapes).to(device)
        self.VQC_input = qml.qnn.TorchLayer(self.qlayer_input, weight_shapes).to(device)
        self.VQC_update = qml.qnn.TorchLayer(self.qlayer_update, weight_shapes).to(device)
        self.VQC_output = qml.qnn.TorchLayer(self.qlayer_output, weight_shapes).to(device)

        # self.VQC = {
        #     "forget": qml.qnn.TorchLayer(self.qlayer_forget, weight_shapes).to(device),
        #     "input": qml.qnn.TorchLayer(self.qlayer_input, weight_shapes).to(device),
        #     "update": qml.qnn.TorchLayer(self.qlayer_update, weight_shapes).to(device),
        #     "output": qml.qnn.TorchLayer(self.qlayer_output, weight_shapes).to(device),
        # }
        # self.clayer_out = torch.nn.Linear(self.n_qubits, self.hidden_size)

        # Turn off classical parameters to see if they're the problem
        # for out_param in self.clayer_out.parameters():
        #   out_param.requires_grad = False
        # for in_param in self.clayer_in.parameters():
        #   in_param.requires_grad = False


    def forward(self, x, init_states=None):
        """
        x.shape is (batch_size, seq_length, feature_size)
        recurrent_activation -> sigmoid
        activation -> tanh
        """
        # Automatically assumes single batch
        x = x.to(self.device)
        # if len(x.shape) == 2:
            # x = x.reshape(1, x.shape[0], x.shape[1])
        seq_length = x.size()
        
        # if self.batch_first is True:
            # batch_size, seq_length, features_size = x.size()
        # else:
            # seq_length, batch_size, features_size = x.size()

        hidden_seq = []
        if init_states is None:
            h_t = torch.zeros(self.hidden_size, device=self.device) # hidden state (output)
            c_t = torch.zeros(self.hidden_size, device=self.device) # cell state
        else:
            # for now we ignore the fact that in PyTorch you can stack multiple RNNs
            # so we take only the first elements of the init_states tuple init_states[0][0], init_states[1][0]
            h_t, c_t = init_states
            # h_t = h_t[0]
            # c_t = c_t[0]

        # for t in range(seq_length):
            # get features from the t-th element in seq, for all entries in the batch
            # x_t = x[:, t, :]

            # Concatenate input and hidden state
        y_t = torch.cat((h_t, x), dim=0).float().to(device)

        # match qubit dimension
        # y_t = self.clayer_in(v_t).to(self.device)
        
        f_t = torch.sigmoid(self.VQC_forget(y_t).to(self.device))  # forget block
        i_t = torch.sigmoid(self.VQC_input(y_t).to(self.device)) # input block
        g_t = torch.tanh(self.VQC_update(y_t).to(self.device))  # update block
        o_t = torch.sigmoid(self.VQC_output(y_t).to(self.device))
        # ).to(self.device)  # output block

        c_t = (f_t * c_t) + (i_t * g_t)
        h_t = o_t * torch.tanh(c_t)

        hidden_seq.append(h_t.unsqueeze(0))

        hidden_seq = torch.cat(hidden_seq, dim=0)
        hidden_seq = hidden_seq.transpose(0, 1).contiguous()

        h_t, c_t = h_t.float(), c_t.float()

        if self.dropout:
          F.dropout(h_t, self.dropout, inplace=True)

        if self.return_state:
            if self.return_sequences:
                return hidden_seq, (h_t, c_t)
            else:
                return (h_t, c_t)
        else:
            if self.return_sequences:
                return hidden_seq
            else:
                return h_t
    
    def predict(self, x, init_states=None):
        return self.forward(x, init_states)

In [5]:
class QLSTM(nn.Module):

  def __init__(self, input_size: int, hidden_size: int, n_layers: int, n_qubits: list, n_qlayers: list, dropouts: list, backend="default.qubit", device="cpu"):
    super(QLSTM, self).__init__()
    self.models = nn.ModuleList()
    self.n_layers = n_layers
    for i in range(self.n_layers):
      self.models.append(
          QLSTMCell(input_size, hidden_size, n_qubits[i], n_qlayers[i], dropouts[i], backend=backend, device=device)
      )
    
  def forward(self, note_sequences):
    # A tuple of (h_t, c_t)
    outputs = []
    h_t_c_t = None
    for i in range(self.n_layers):
      h_t_c_t = self.models[i](note_sequences[i], h_t_c_t)
      # Only output c_t
      outputs.append(h_t_c_t[1])
    
    # print(outputs)
    return torch.stack(outputs)

  def predict(self, note_sequences):
    return self.forward(note_sequences)


In [33]:
def post_processing(ct_list, starting_i):
  i = starting_i
  for j in range(starting_i, starting_i + ct_list.shape[0]):
    ct_list[j-i] = (ct_list[j-i] * float(midi.input_norms[j])).long()
    ct_list[j-i] = torch.mean(ct_list[j-i]).long()
  
  return ct_list[:, 0]

In [52]:
# LSTMusic.py

# import torch
# import torch.nn as nn
# from torch import optim
# import torch.nn.functional as F

# from QLTSM.qlstm import QLSTM

import numpy as np


class LSTMusic(nn.Module):
    def __init__(
        self,
        n_qlayers=1,
        n_layers=1,
        dropout=0.3,
        n_vocab=None,
        input_dim=1,
        hidden_dim=512,
        n_qubits=4,
        backend="default.qubit",
        device="cpu",
    ):
        super(LSTMusic, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        if n_qubits > 0:
            print(f"Generator will use Quantum LSTM running on backend {backend}")
            r_n = range(n_layers)
            n_qubits = [n_qubits for _ in r_n]
            n_qlayers = [n_qlayers for _ in r_n]
            dropouts = [dropout for _ in r_n]
            self.model = QLSTM(input_dim, hidden_dim, n_layers, n_qubits, n_qlayers, dropouts, device=device).to(device)
        else:
            print("Generator will use Classical LSTM")
            self.model = nn.LSTM(input_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space

    def forward(self, note_sequences):
        ct_list = self.model(note_sequences)
        return torch.abs(ct_list)
        # scores = []
        # for c_t in ct_list:
          # scores.append(F.log_softmax(c_t, dim=1))
        # ct_list = torch.stack(scores)
        # return ct_list.reshape(ct_list.shape[0], ct_list.shape[2])

    def predict(self, note_sequences):
        return self.forward(note_sequences)

    def train(
        self,
        mode=True,
        inputs=None,
        outputs=None,
        n_epochs=None,
        cutoff: int = None,
        learning_rate=0.1,
    ):
        # Same as categorical cross entropy, who would've thought?!
        if mode == False:
            return
        loss_function = nn.MSELoss()
        # loss_function = nn.NLLLoss()
        optimizer = optim.SGD(filter(lambda p: p.requires_grad, self.model.parameters()), lr=learning_rate)

        if cutoff:
            inputs = inputs[:cutoff]
            outputs = outputs[:cutoff]

        history = {"loss": []}

        midi_data = list(zip(inputs, outputs))

        for epoch in range(n_epochs):
            counter = 0
            losses = []

            for i in range(0, len(midi_data) - self.n_layers, self.n_layers):
              data = midi_data[i:i+self.n_layers]
              note_seqs = [datum[0] for datum in data]
              next_notes = torch.stack([datum[1] for datum in data])

              optimizer.zero_grad()
              c_t_list = self(note_seqs)
              c_t_list = post_processing(c_t_list, i)

              loss = loss_function(c_t_list, next_notes.float())
              loss.backward()
              optimizer.step()
              losses.append(float(loss))

              if counter % 5 == 0:
                print(f"On datapoint #{counter} out of {cutoff}")
              counter += 1

            avg_loss = np.mean(losses)
            history["loss"].append(avg_loss)
            print("Epoch {} / {}: Loss = {:.3f}".format(epoch + 1, n_epochs, avg_loss))
        return history

In [39]:
n_qubits = 6
seq_length = 2**n_qubits - n_qubits
n_epochs = 1
cutoff = 200
n_layers = 4
n_qlayers = 2

model_name = f"lstm{n_layers}-seq{seq_length}-cut{cutoff}-epcs{n_epochs}-qu{n_qubits}-nq{n_qlayers}"
model_str = f"{model_name}.pt"

In [10]:
print("Initialized Midi")
midi = Midi(seq_length, device)

Initialized Midi


C:\Users\TheRe\AppData\Local\Temp/ipykernel_2796/440410498.py:79: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.input_norms = torch.tensor(torch.linalg.norm(network_input, axis=1))


Input shape: torch.Size([44798, 58])
Output shape: torch.Size([44798])


In [40]:
print("Initialized LSTM")
lstm = LSTMusic(n_qubits=n_qubits, n_qlayers=n_qlayers, n_layers=n_layers, hidden_dim=n_qubits, device=device).to(device)

Initialized LSTM
Generator will use Quantum LSTM running on backend default.qubit
weight_shapes = (n_qlayers, n_qubits) = (2, 6)
weight_shapes = (n_qlayers, n_qubits) = (2, 6)
weight_shapes = (n_qlayers, n_qubits) = (2, 6)
weight_shapes = (n_qlayers, n_qubits) = (2, 6)


In [ ]:
# for param in lstm.model.models[0].clayer_in.parameters():
#   print(param.requires_grad)

False
False


In [12]:
def print_parameters():
  for name, param in lstm.named_parameters():
      if param.requires_grad:
          print(name, param.data)

In [41]:
# TODO: Separate input data to test/train

if Path(model_str).is_file():
    print("Loading model")
    lstm.load_state_dict(torch.load(model_str))
    lstm.eval()
    # lstm = torch.load(model_str)
else:
    print("Training LSTM")
    train_history = lstm.train(
        True, midi.network_input, midi.network_output, n_epochs=n_epochs, cutoff=cutoff
    )
    torch.save(lstm.state_dict(), model_str)

Training LSTM
On datapoint #0 out of 200
On datapoint #5 out of 200
On datapoint #10 out of 200
On datapoint #15 out of 200
On datapoint #20 out of 200
On datapoint #25 out of 200
On datapoint #30 out of 200
On datapoint #35 out of 200
On datapoint #40 out of 200
On datapoint #45 out of 200
Epoch 1 / 1: Loss = 11886.245


In [74]:
def generate_notes(self, network_input, int_to_note, n_vocab, n_notes, n_layers):
        """Generate notes from the neural network based on a sequence of notes"""
        # pick a random sequence from the input as a starting point for the prediction
        with torch.no_grad():
            req_size = n_notes//n_layers
            start = np.random.randint(0, len(network_input) - req_size)

            # pattern = network_input[start]
            prediction_output = []

            # generate n_notes
            for i in range(start, start + n_notes, n_layers):
                # print(network_input[i:i+n_layers].shape)
                prediction_input = network_input[i:i+n_layers]
                # prediction_input = pattern.clone().detach().reshape(1, len(pattern), 1)
                # prediction_input = prediction_input / float(n_vocab)

                ct_list = self.forward(prediction_input)
                ct_list = post_processing(ct_list, i)
                # print(ct_list)
                # print("Gib")
                # print(ct_list)

                for prediction in ct_list:
                  index = torch.mean(prediction)
                  # print(prediction, index)
                  result = int_to_note.get(int(index), '0')
                  prediction_output.append(result)

                # added_index = (index / n_vocab).reshape(1, 1)
                # pattern = torch.cat((pattern, added_index), 0)
                # pattern.append(index)
                # pattern = pattern[1 : len(pattern)]

            return prediction_output

In [ ]:
n_notes = 20
req_size = n_notes//n_layers
start = np.random.randint(0, len(midi.network_input) - req_size)
with torch.no_grad():
  prediction_input = midi.network_input[start:start+n_layers]
  ct_list = lstm.model.predict(prediction_input)
  print(ct_list[0])

In [ ]:
print_parameters()

In [75]:
print("Generating notes")
notes = generate_notes(
    lstm, midi.network_input, midi.int_to_note, midi.n_vocab, n_notes=20, n_layers=n_layers
)
notes

Generating notes


['6.7.11',
 '7.11.0',
 '308',
 '308',
 '308',
 '9.1.4',
 'G#3',
 '3.5.8',
 '9',
 '8.11',
 '8.10.0',
 '9.0.2',
 '11.2',
 '308',
 '7.8.10.2',
 '4',
 '7.9.11',
 '5.9.11',
 '6.9',
 'E-6']

In [76]:
print("Saving as MIDI file.")
midi.create_midi_from_model(notes, f"{model_name}_generated_2.mid")

Saving as MIDI file.
